# 01 – Veri Keşfi ve Dominant Renk Çıkarımı

Bu defterde veri seti üzerinde temel keşif (EDA) yapacak ve her ürün için dominant renkleri çıkaracağız.

In [ ]:
# Gerekli kütüphaneler
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

data_root = 'data/deepfashion2'  # gerekirse güncelle


In [ ]:
# Örnek görsel yollarını listeleme (örnek)
image_paths = []
for root, dirs, files in os.walk(data_root):
    for fname in files:
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(root, fname))

print('Toplam örnek görsel:', len(image_paths))
image_paths[:5]

In [ ]:
# Örnek bir görseli gösterme
img = Image.open(image_paths[0]).convert('RGB')
plt.imshow(img)
plt.axis('off')

In [ ]:
# Dominant renkleri çıkaran yardımcı fonksiyon
def extract_dominant_colors(image, n_colors=5):
    img = image.resize((128, 128))  # hız için küçült
    arr = np.asarray(img).reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_colors, random_state=42)
    kmeans.fit(arr)
    colors = kmeans.cluster_centers_
    return colors  # shape: (n_colors, 3)


In [ ]:
# Basit bir döngü ile subset için renkleri çıkarma (örnek)
records = []
subset_paths = image_paths[:500]  # küçük bir subset

for idx, p in enumerate(subset_paths):
    img = Image.open(p).convert('RGB')
    colors = extract_dominant_colors(img, n_colors=5)
    flat_colors = colors.flatten()  # 5*3 = 15 boyut
    records.append({
        'product_id': idx,
        'image_path': p,
        **{f'c{i}': v for i, v in enumerate(flat_colors)}
    })

df_colors = pd.DataFrame.from_records(records)
df_colors.head()

In [ ]:
# Sonuçları kaydet
os.makedirs('data/processed', exist_ok=True)
df_colors.to_csv('data/processed/colors.csv', index=False)
print('Kaydedildi: data/processed/colors.csv')